# Experimentación - Asistente Académico RAG

Notebook para probar y experimentar con el sistema RAG

## 1. Imports y Setup

In [ ]:
import sys
sys.path.append('..')

from asistente import AsistenteAcademico
from evaluacion.metricas import EvaluadorAsistente
import json

## 2. Inicializar Asistente

In [ ]:
# Crear instancia del asistente
asistente = AsistenteAcademico(modelo_llama="llama2:7b")

## 3. Cargar Documentos

In [ ]:
# Lista de PDFs a procesar
pdfs = [
    "../documentos/apuntes_ia.pdf",
    # Agregar más PDFs aquí
]

# Cargar documentos (esto puede tomar unos minutos)
asistente.cargar_documentos(pdfs)

## 4. Probar Consultas

In [ ]:
# Hacer una pregunta de prueba
pregunta = "¿Qué es RAG?"
resultado = asistente.consultar(pregunta)

print("Respuesta:")
print(resultado["respuesta"])
print("\n" + "="*60)

# Mostrar fuentes
asistente.mostrar_fuentes(resultado["fuentes"])

## 5. Evaluación de Calidad

In [ ]:
# Crear evaluador
evaluador = EvaluadorAsistente(asistente)

# Cargar conjunto de pruebas
with open('../evaluacion/test_preguntas.json', 'r', encoding='utf-8') as f:
    test_set = json.load(f)

# Ejecutar evaluación
resultados = evaluador.evaluar_conjunto_completo(test_set)

print(f"Tiempo promedio de respuesta: {resultados['tiempo_promedio']:.2f}s")
if 'rouge1_promedio' in resultados:
    print(f"ROUGE-1 Score: {resultados['rouge1_promedio']:.3f}")

## 6. Experimentar con Parámetros

In [ ]:
# Probar diferentes valores de top-k
pregunta_test = "¿Cuáles son los objetivos del proyecto?"

for k in [1, 3, 5]:
    print(f"\n{'='*60}")
    print(f"Top-K = {k}")
    print("="*60)
    
    # Aquí puedes modificar el retriever para usar diferentes k
    # asistente.vectorstore.as_retriever(search_kwargs={"k": k})
    
    resultado = asistente.consultar(pregunta_test)
    print(resultado["respuesta"][:200] + "...")

## 7. Análisis de Fragmentos Recuperados

In [ ]:
# Analizar qué fragmentos se recuperan para una consulta
pregunta = "Explica las técnicas de IA utilizadas"
resultado = asistente.consultar(pregunta)

print(f"Fragmentos recuperados: {len(resultado['fuentes'])}\n")

for i, doc in enumerate(resultado['fuentes'], 1):
    print(f"[{i}] Página {doc.metadata.get('page', '?')}")
    print(f"Contenido: {doc.page_content[:150]}...")
    print("-" * 60)

## 8. Notas y Observaciones